<a href="https://colab.research.google.com/github/omkarpat/eeg-mood-classification/blob/master/Baseline_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_images/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [0]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D

In [17]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
#relaxed_data_array = np.load("relaxed_array_updated_spectrograms_final.npy")
for filename in os.listdir(base_path):
  if "concentration_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "relaxed_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)

concentration_array_updated_spectrograms_1.npy
concentration_array_updated_spectrograms_2.npy
concentration_array_updated_spectrograms_3.npy
concentration_array_updated_spectrograms_4.npy
concentration_array_updated_spectrograms_5.npy
concentration_array_updated_spectrograms_6.npy
relaxed_array_updated_spectrograms_1.npy
relaxed_array_updated_spectrograms_2.npy
relaxed_array_updated_spectrograms_3.npy
relaxed_array_updated_spectrograms_4.npy
relaxed_array_updated_spectrograms_5.npy


In [18]:
print(concentration_data_array.shape, relaxed_data_array.shape)

(2160, 29, 43, 42) (1800, 29, 43, 42)


In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [21]:
Y = np.concatenate((np.zeros((2160,1)),np.ones((1800,1))))
print(X.shape, Y.shape)

(3960, 29, 43, 42) (3960, 1)


In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(2861, 29, 43, 42) (594, 29, 43, 42) (505, 29, 43, 42) (2861, 1) (594, 1) (505, 1)


In [0]:
model2 = Sequential()
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape=(29, 43, 42)))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.0000001, momentum=0.7)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=8, shuffle=True)

Train on 2861 samples, validate on 505 samples
Epoch 1/50
2861/2861 [==============================] - 57s 20ms/step - loss: 5.6097 - acc: 0.5341 - val_loss: 5.8282 - val_acc: 0.5307
Epoch 2/50
2861/2861 [==============================] - 57s 20ms/step - loss: 5.0841 - acc: 0.5481 - val_loss: 4.9180 - val_acc: 0.5465
Epoch 3/50
1880/2861 [==================>...........] - ETA: 18s - loss: 5.0302 - acc: 0.5553